# Future Network Constrained Linear Optimal Power Flow with Floating Wind & Marine

PyPSA-GB can model the GB power system  by solving a network constrained Linear Optimal Power Flow (LOPF) problem. This notebook shows the example application of a future period, and disaggregates the results by novel offshore renewables: floating wind, wave power and tidal stream.  Tidal lagoon power is also disaggregated.

In [1]:
import os
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
src_path = os.environ.get('PROJECT_SRC')
os.chdir(src_path)

In [2]:
import pypsa
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs

import data_reader_writer
import generators

## Setting up simulation

Set the required inputs for the LOPF: the start, end and year of simulation, and the timestep.

In [3]:
# write csv files for import
start = '2050-06-01 00:00:00'
end = '2050-06-01 23:30:00'
# year of simulation
year = int(start[0:4])
# time step as fraction of hour
time_step = 1.0

Choose from one of the National Grid Future Energy Scenarios.

In [4]:
scenario = 'Leading The Way'
# scenario = 'Consumer Transformation'
# scenario = 'System Transformation'
# scenario = 'Steady Progression'

Choose a baseline year (from 2010-2020). The baseline year determines which historical load profile and weather dataset is used for the future year modelled. The National Grid FES modellers used 2012 as their baseline year.

In [5]:
year_baseline = 2012

data_reader_writer is a script written to read in data from the various sources and write csv files in the format required for populating a PyPSA network object

In [6]:
# floating wind TRUE/FALSE input...

# if floating wind True, then set 'merge_generators=False', in data_writer function else set merge_generators=True

# Setting merge_generators argument FALSE manually yields generators.csv with types and sites denoted. Excellent!

In [6]:
data_reader_writer.data_writer(start, end, time_step, year, year_baseline=year_baseline,
                               scenario=scenario, merge_generators=False)

C:\Users\s1100626\Anaconda3\envs\PyPSA-GB\lib\site-packages\pandas\core\arrays\datetimes.py:741: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  warnings.warn(
C:\Users\s1100626\Anaconda3\envs\PyPSA-GB\lib\site-packages\pandas\core\indexing.py:1667: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = value
C:\Users\s1100626\Documents\GitHub\PyPSA-GB\PyPSA-GB\interconnectors.py:201: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df_FES = df_FES[~df_FES.Variable.str.contains('(TWh)')]


In [7]:
network = pypsa.Network()

network.import_from_csv_folder('LOPF_data')

Importing PyPSA from older version of PyPSA than current version.
Please read the release notes at https://pypsa.readthedocs.io/en/latest/release_notes.html
carefully to prepare your network for import.
Currently used PyPSA version [0, 19, 2], imported network file PyPSA version None.

INFO:pypsa.components:Applying weightings to all columns of `snapshot_weightings`
INFO:pypsa.io:Imported network LOPF_data has buses, generators, lines, links, loads, storage_units


Lines need to be scaled up to accomadate for future generation, and specific analysis will be done on this in a later notebook.
Note: interconnects are links in future, so don't need to be selective here (as was required in historical simulation).

In [9]:
contingency_factor = 4
network.lines.s_max_pu *= contingency_factor

## Running the optimisation

In [10]:
network.lopf(network.snapshots, solver_name="gurobi", pyomo=False)

INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 1.84s
INFO:pypsa.linopf:Solve linear problem using Gurobi solver


Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-19
Error reading LP format file C:\Users\s1100626\AppData\Local\Temp\pypsa-problem-zf2jlg1v.lp at line 287887
Unrecognized constraint RHS or sense
Neighboring tokens: " <= +nan c70230: +1.000000 x1230 <= +1.133519 "

Unable to read file


GurobiError: Unable to read model

## Power output by generation type

Group the generators by the carrier, and print their summed power outputs over the simulation period.

In [ ]:
df = pd.DataFrame(network.generators) 
    
# saving the dataframe 
df.to_csv('network.generators.csv') 

In [ ]:
# df = pd.DataFrame(network.generators_t.p) 
    
# # saving the dataframe 
# df.to_csv('network.generators_t.p.csv') 

In [ ]:
network.generators

In [ ]:
type(network.generators)

In [ ]:
# read network.generators
# in network.generators, offshore wind is already grouped by BUS - this means that new carrier type is required...?
# find floating wind sites: Generator = I, E, F, G, NE8, NE7, E3, E2, NE1, E1, NE2, NE3, NE6, N2, N3
# https://datagy.io/pandas-replace-values/

In [ ]:
p_by_carrier = network.generators_t.p.groupby(
    network.generators.carrier, axis=1).sum()

storage_by_carrier = network.storage_units_t.p.groupby(
    network.storage_units.carrier, axis=1).sum()

# to show on graph set the negative storage values to zero
storage_by_carrier[storage_by_carrier < 0] = 0

p_by_carrier = pd.concat([p_by_carrier, storage_by_carrier], axis=1)

print(network.links_t.p0)
imp = network.links_t.p0.copy()
imp[imp < 0] = 0
imp['Interconnectors Import'] = imp.sum(axis=1)
interconnector_import = imp[['Interconnectors Import']]
print(interconnector_import)

p_by_carrier = pd.concat([p_by_carrier, interconnector_import], axis=1)

exp = network.links_t.p0.copy()
exp[exp > 0] = 0
exp['Interconnectors Export'] = exp.sum(axis=1)
interconnector_export = exp[['Interconnectors Export']]
print(interconnector_export)

# group biomass stuff
p_by_carrier['Biomass'] = (
    p_by_carrier['Biomass (dedicated)'] + p_by_carrier['Biomass (co-firing)'] +
    p_by_carrier['Landfill Gas'] + p_by_carrier['Anaerobic Digestion'] +
    p_by_carrier['Sewage Sludge Digestion'])

# rename the hydro bit
p_by_carrier = p_by_carrier.rename(
    columns={'Large Hydro': 'Hydro'})
p_by_carrier = p_by_carrier.rename(
    columns={'Interconnector': 'Interconnectors Import'})

p_by_carrier

In [ ]:
p_by_type = network.generators_t.p.groupby(
    network.generators.type, axis=1).sum()

storage_by_type = network.storage_units_t.p.groupby(
    network.storage_units.type, axis=1).sum()

# to show on graph set the negative storage values to zero
storage_by_type[storage_by_type < 0] = 0

p_by_type = pd.concat([p_by_type, storage_by_type], axis=1)

print(network.links_t.p0)
imp = network.links_t.p0.copy()
imp[imp < 0] = 0
imp['Interconnectors Import'] = imp.sum(axis=1)
interconnector_import = imp[['Interconnectors Import']]
print(interconnector_import)

p_by_type = pd.concat([p_by_type, interconnector_import], axis=1)

exp = network.links_t.p0.copy()
exp[exp > 0] = 0
exp['Interconnectors Export'] = exp.sum(axis=1)
interconnector_export = exp[['Interconnectors Export']]
print(interconnector_export)

# group biomass stuff
p_by_type['Biomass'] = (
    p_by_type['Biomass (dedicated)'] + p_by_type['Biomass (co-firing)'] +
    p_by_type['Landfill Gas'] + p_by_type['Anaerobic Digestion'] +
    p_by_type['Sewage Sludge Digestion'])

# rename the hydro bit
p_by_type = p_by_type.rename(
    columns={'Large Hydro': 'Hydro'})
p_by_type = p_by_type.rename(
    columns={'Interconnector': 'Interconnectors Import'})

p_by_type

In [ ]:
df = pd.DataFrame(p_by_carrier) 
    
# saving the dataframe 
df.to_csv('p_by_carrier.csv') 

In [ ]:
df = pd.DataFrame(p_by_type) 
    
# saving the dataframe 
df.to_csv('p_by_type.csv') 

Graph the power output of the different generation types...

In [ ]:
read in csv containing generation types

In [ ]:
cols = ['Nuclear', 'Biomass',
        'EfW Incineration', 'Oil', 'Natural Gas',
        'Hydrogen', 'CCS Gas', 'CCS Biomass','Interconnectors Import',
        'Pumped Storage Hydroelectric', 'Hydro',
        'Battery', 'Compressed Air', 'Liquid Air',
        'Wind Offshore', 'Wind Onshore', 'Solar Photovoltaics',
        'Tidal lagoon', 'Tidal stream', 'Wave power', 'Unmet Load'
        ]

# types alphabetically are:




p_by_carrier = p_by_carrier[cols]

# edited such that all sources are shown regardless of power contribution
p_by_carrier.drop(
    (p_by_carrier.max()[p_by_carrier.max() < 0.0]).index,
    axis=1, inplace=True)


colors = {'Coal': 'dimgrey',
          'Diesel/Gas oil': 'lightgrey',
          'Diesel/gas Diesel/Gas oil': 'lightgrey',
          'Oil': 'red',
          'Unmet Load': 'black',
          'Anaerobic Digestion': 'darkgoldenrod',
          'EfW Incineration': 'chocolate',
          'Sewage Sludge Digestion': 'saddlebrown',
          'Landfill Gas': 'olive',
          'Biomass (dedicated)': 'olivedrab',
          'Biomass (co-firing)': 'yellowgreen',
          'Biomass': 'greenyellow',
          'CCS Biomass': 'darkolivegreen',
          'Interconnectors Import': 'palevioletred',
          'Interconnectors Export': 'crimson',          
          'Sour gas': 'darkred',
          'Natural Gas': 'coral',
          'CCS Gas': 'lightcoral',
          'Hydrogen': 'paleturquoise',
          'Nuclear': 'lime',
          'Wave power': 'steelblue',
          'Tidal lagoon': 'mediumblue',
          'Tidal stream': 'midnightblue',
          'Hydro': 'teal',
          'Large Hydro': 'darkturquoise',
          'Small Hydro': 'turquoise',
          'Pumped Storage Hydroelectric': 'deepskyblue',
          'Battery': 'mediumorchid',
          'Compressed Air': 'plum',
          'Liquid Air': 'thistle',
          'Floating Wind': 'royalblue',
          'Wind Offshore': 'cornflowerblue',
          'Wind Onshore': 'mediumseagreen',
          'Solar Photovoltaics': 'yellow'}

fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)
(p_by_carrier / 1e3).plot(
    kind='area', ax=ax, linewidth=0,
    color=[colors[col] for col in p_by_carrier.columns])

# stacked area plot of negative values, prepend column names with '_' such that they don't appear in the legend
(interconnector_export / 1e3).plot.area(ax=ax, stacked=True, linewidth=0.)
# rescale the y axis
ax.set_ylim([(interconnector_export / 1e3).sum(axis=1).min(), (p_by_carrier / 1e3).sum(axis=1).max()])

# Shrink current axis's height by 10% on the bottom
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.1,
                 box.width, box.height * 0.9])

# Put a legend below current axis
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=5)

ax.set_ylabel('GW')

ax.set_xlabel('')

## Calculating emissions

Calculate the emissions from each carrier according to the power output...

## Plotting storage

Graph the pumped hydro dispatch and state of charge...

In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)

p_storage = network.storage_units_t.p.sum(axis=1)
state_of_charge = network.storage_units_t.state_of_charge.sum(axis=1)
p_storage.plot(label="Pumped hydro dispatch", ax=ax, linewidth=3)
state_of_charge.plot(label="State of charge", ax=ax, linewidth=3)

ax.legend()
ax.grid()
ax.set_ylabel("MWh")
ax.set_xlabel("")

## Plotting line loading

Look at the line loading stats and graph...

In [ ]:
now = network.snapshots[139]

print("With the linear load flow, there is the following per unit loading:")
loading = network.lines_t.p0.loc[now] / network.lines.s_nom
loading.describe()

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={"projection": ccrs.PlateCarree()})
fig.set_size_inches(15, 17)

network.plot(ax=ax, line_colors=abs(loading), line_cmap=plt.cm.jet, title="Line loading")

## Plotting locational marginal prices

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={"projection": ccrs.PlateCarree()})
fig.set_size_inches(20, 10)

network.plot(ax=ax, line_widths=pd.Series(0.5, network.lines.index))
plt.hexbin(network.buses.x, network.buses.y,
           gridsize=20,
           C=network.buses_t.marginal_price.loc[now],
           cmap=plt.cm.jet)

# for some reason the colorbar only works with graphs plt.plot
# and must be attached plt.colorbar

cb = plt.colorbar()
cb.set_label('Locational Marginal Price (£/MWh)')

In [ ]:
network.buses_t.marginal_price

## Plotting curtailment

In [ ]:
carrier = "Wind Onshore"

capacity = network.generators.groupby("carrier").sum().at[carrier, "p_nom"]
p_available = network.generators_t.p_max_pu.multiply(network.generators["p_nom"])
p_available_by_carrier = p_available.groupby(network.generators.carrier, axis=1).sum()
p_curtailed_by_carrier = p_available_by_carrier - p_by_carrier
p_df = pd.DataFrame({carrier + " available": p_available_by_carrier[carrier],
                     carrier + " dispatched": p_by_carrier[carrier],
                     carrier + " curtailed": p_curtailed_by_carrier[carrier]})

p_df[carrier + " capacity"] = capacity
p_df["Wind Onshore curtailed"][p_df["Wind Onshore curtailed"] < 0.] = 0.
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)
p_df[[carrier + " dispatched", carrier + " curtailed"]].plot(kind="area", ax=ax, linewidth=0)
p_df[[carrier + " available", carrier + " capacity"]].plot(ax=ax, linewidth=0)

ax.set_xlabel("")
ax.set_ylabel("Power [MW]")
ax.legend()